In [2]:
import csv
data = {}
with open('data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]

In [3]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
import os
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [6]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/', emotion))
  os.mkdir(os.path.join('master_data/testing/', emotion))

In [7]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('images', image)
    dest = os.path.join('master_data/training', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('images', image)
    dest = os.path.join('master_data/testing', emotion, image)
    copyfile(source, dest)

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(512, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)             

In [ ]:
train_dir = '/master_data/training'
test_dir = '/master_data/testing'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 128
)
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta = 0.01)

In [ ]:
model.fit(train_generator, epochs = 10, verbose = 1, validation_data = test_generator, callbacks = [es])

Epoch 1/10
86/86 [==============================] - 18s 213ms/step - loss: 0.5962 - acc: 0.7944 - val_loss: 1.3076 - val_acc: 0.6200
Epoch 2/10
86/86 [==============================] - 18s 209ms/step - loss: 0.5991 - acc: 0.7917 - val_loss: 1.0591 - val_acc: 0.6627
Epoch 3/10
86/86 [==============================] - 18s 211ms/step - loss: 0.5643 - acc: 0.8059 - val_loss: 1.2807 - val_acc: 0.6375
Epoch 4/10
86/86 [==============================] - 18s 208ms/step - loss: 0.5351 - acc: 0.8102 - val_loss: 1.3568 - val_acc: 0.6524
